# Conhecendo os dados

### Desafio: utilizando parâmetros do json_normalize

Ao normalizar o objeto JSON durante a aula passamos simplesmente ```pd.json_normalize(<dados>)``` mas o método json_normalize possui diversos parâmetros para trabalhar com estruturas de dados mais complexas.

Você recebeu a tarefa de normalizar o seguinte arquivo JSON chamado ```“informacoes.json”``` referente a algumas informações de identificação de um cliente. O arquivo possui o seguinte conteúdo:

```python
{
  "nome": "João",
  "idade": 28,
  "enderecos": [
    {
      "tipo": "casa",
      "rua": "Rua A",
      "numero": 123,
      "cidade": "São Paulo"
    },
    {
      "tipo": "trabalho",
      "rua": "Rua B",
      "numero": 456,
      "cidade": "Rio de Janeiro"
    }
  ]
}
```

O resultado final deve ser um Pandas DataFrame que possui a seguinte estrutura:

![tabela](assets/pandas_limpeza_tratamento_de_dados_aula1.png)

Como podemos fazer isso?

In [1]:
import pandas as pd
import json

In [2]:
url = 'pandas-dados/informacoes.json'

In [3]:
dados = pd.read_json(url)
dados.head()

,nome,idade,enderecos
0,João,28,"{'tipo': 'casa', 'rua': 'Rua A', 'numero': 123..."
1,João,28,"{'tipo': 'trabalho', 'rua': 'Rua B', 'numero':..."


In [4]:
pd.json_normalize(dados['enderecos'])

,tipo,rua,numero,cidade
0,casa,Rua A,123,São Paulo
1,trabalho,Rua B,456,Rio de Janeiro


In [5]:
with open(url) as arquivo:
    json_bruto = json.load(arquivo)

In [6]:
dados_normlizados = pd.json_normalize(json_bruto, record_path='enderecos',meta=["nome", "idade"])
dados_normlizados.head()

,tipo,rua,numero,cidade,nome,idade
0,casa,Rua A,123,São Paulo,João,28
1,trabalho,Rua B,456,Rio de Janeiro,João,28


### Desafio: tratando os dados

Você recebeu e fez o download o arquivo JSON chamado “dados.json” que possui o seguinte conteúdo:

```json
{
  "pessoas": [
    {
      "nome": "João",
      "idade": "25",
      "endereco": {
        "rua": "Rua A",
        "numero": 123,
        "cidade": "São Paulo"
      },
      "telefones": [
        "11 1111-1111",
        "11 2222-2222"
      ]
    },
    {
      "nome": "Maria",
      "idade": 30,
      "endereco": {
        "rua": "",
        "numero": 456,
        "cidade": "Rio de Janeiro"
      },
      "telefones": [
        "21 3333-3333"
      ]
    }
  ]
}
```

Sua tarefa é realizar alguns tratamentos e deixar esse arquivo JSON como um Pandas DataFrame, sem dados vazios e com a tipagem das colunas de forma correta. Como isso pode ser feito?

In [7]:
import pandas as pd
import json

In [8]:
url = 'pandas-dados/dados.json'

In [9]:
with open(url) as arquivo:
    dados = json.load(arquivo)

In [10]:
dados

{'pessoas': [{'nome': 'João',
   'idade': '25',
   'endereco': {'rua': 'Rua A', 'numero': 123, 'cidade': 'São Paulo'},
   'telefones': ['11 1111-1111', '11 2222-2222']},
  {'nome': 'Maria',
   'idade': 30,
   'endereco': {'rua': '', 'numero': 456, 'cidade': 'Rio de Janeiro'},
   'telefones': ['21 3333-3333']}]}

In [11]:
df = pd.json_normalize(dados, record_path=['pessoas'])
df

,nome,idade,telefones,endereco.rua,endereco.numero,endereco.cidade
0,João,25,"[11 1111-1111, 11 2222-2222]",Rua A,123,São Paulo
1,Maria,30,[21 3333-3333],,456,Rio de Janeiro


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nome             2 non-null      object
 1   idade            2 non-null      object
 2   telefones        2 non-null      object
 3   endereco.rua     2 non-null      object
 4   endereco.numero  2 non-null      int64 
 5   endereco.cidade  2 non-null      object
dtypes: int64(1), object(5)
memory usage: 228.0+ bytes


analisando as informacoes acima, preciso corrigir:

1. acertar o tipo da coluna idade
2. retirar a linha com dados nulos

In [13]:
df['idade'] = df['idade'].astype(int)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nome             2 non-null      object
 1   idade            2 non-null      int32 
 2   telefones        2 non-null      object
 3   endereco.rua     2 non-null      object
 4   endereco.numero  2 non-null      int64 
 5   endereco.cidade  2 non-null      object
dtypes: int32(1), int64(1), object(4)
memory usage: 220.0+ bytes


In [15]:
filtro_nulas = df['endereco.rua'] != ''

In [16]:
df_filtrado = df[filtro_nulas].copy()
df_filtrado

,nome,idade,telefones,endereco.rua,endereco.numero,endereco.cidade
0,João,25,"[11 1111-1111, 11 2222-2222]",Rua A,123,São Paulo
